In [26]:
import requests
import os
import json
import pandas as pd
import csv
import datetime as dt
import dateutil.parser
import unicodedata
import time

In [27]:
os.environ['TOKEN'] = f'AAAAAAAAAAAAAAAAAAAAABoCXgEAAAAA%2Fywpstn66jWtU%2BUPxEvewEBd9gs%3DCjyeAelogvSYHAFTclWfAQrKvMs7EEoZyYSOHTdc4M0MPFmB6W'

def auth():
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(keywords, start_time, end_time=None, max_results=5):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
    
    if not end_time:
        end_time = dt.utc_now().strftime('%Y-%m-%DT%H:%M:%S%Z')
    
    #change params based on the endpoint you are using
    query_params = {'query': ' '.join(keywords),
                    'start_time': start_time,
                    'end_time': end_time,
                    'max_results': max_results,
                    'expansions': 'author_id,referenced_tweets.id',
                    #'tweet.fields': 'attachments,author_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,source,text,withheld',
                    'tweet.fields': 'author_id,created_at,id,lang,source,text,withheld',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'next_token': {}}
    return (search_url, query_params)

In [28]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [29]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keywords = ['AAPL', 'Apple']
start_time = dt.datetime.now(dt.timezone.utc) - dt.timedelta(hours=12)
start_time = start_time.strftime('%Y-%m-%dT%H:%M:%SZ')
end_time = dt.datetime.now(dt.timezone.utc) - dt.timedelta(seconds=30)
end_time = end_time.strftime('%Y-%m-%dT%H:%M:%SZ')
max_results = 10

In [30]:
url = create_url(keywords, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])
print(json.dumps(json_response, indent=4, sort_keys=True))

Endpoint Response Code: 200
{
    "data": [
        {
            "author_id": "15281391",
            "created_at": "2022-01-04T22:30:18.000Z",
            "id": "1478494020443820037",
            "lang": "en",
            "source": "Buffer",
            "text": "$AAPL recently hit new all-time highs, but where is the rest of FAANG? \nhttps://t.co/tYtNgD9S8Y"
        },
        {
            "author_id": "1391167212690591744",
            "created_at": "2022-01-04T22:29:14.000Z",
            "id": "1478493753711243268",
            "lang": "en",
            "referenced_tweets": [
                {
                    "id": "1478360558919368707",
                    "type": "retweeted"
                }
            ],
            "source": "Twitter for iPhone",
            "text": "RT @StockMKTNewz: Apple $AAPL today was granted 2 patents today for a 'Multi-linkage vehicle door hinge\" and a \"Vehicle seat with reclining\u2026"
        },
        {
            "author_id": "98581178984

In [32]:
base_tweet_link = 'https://twitter.com/i/web/status/{}'
links_list = list()
for tweet in json_response['data']:
    links_list.append(base_tweet_link.format(tweet['id']))

print(links_list)

['https://twitter.com/i/web/status/1478494020443820037', 'https://twitter.com/i/web/status/1478493753711243268', 'https://twitter.com/i/web/status/1478493720047599620', 'https://twitter.com/i/web/status/1478492575392694273', 'https://twitter.com/i/web/status/1478492384019234820', 'https://twitter.com/i/web/status/1478492173956005889', 'https://twitter.com/i/web/status/1478492050450493448', 'https://twitter.com/i/web/status/1478491723554717699', 'https://twitter.com/i/web/status/1478491627127816196', 'https://twitter.com/i/web/status/1478491605694885903']
